In [1]:
import itertools as it
import pandas as pd
import requests as rq
from IPython.display import display

In [2]:
users = "../data/users.json"
host = "localhost"
port = "5000"
limit = 10
walks = 1000
walk_length = 100

In [3]:
# Import users
user_df = pd.read_json(users, orient="table")
display(user_df)

,user
2267238,4671229608
1396952,2312687048
5066612,228541032
3807989,946097265625538560
2680420,1018562002678108162
...,...
748630,1597388371
1156305,2696695362
5014030,1330692817
962078,2987955914


In [4]:
def get_recommendations(user_df, walks, walk_length, limit):
    rows = []

    for i, row in user_df.iterrows():
        user = row["user"]
        url = f"http://{host}:{port}/recommendation/salsa/{user}?walks={walks}&walk_length={walk_length}&limit={limit}"
        response = rq.get(url)

        status_code = response.status_code
        duration = response.elapsed.total_seconds()
        recommendations = response.json() if response.status_code == 200 else []
        rows.append((walks, walk_length, limit, url, status_code, duration, user, recommendations))
    
    return pd.DataFrame(rows, columns=["walks", "walk_length", "limit", "url", "status_code", "duration", "user", "recommendations"])

In [5]:
recommendations_df = get_recommendations(user_df, walks, walk_length, limit)

In [6]:
# Save results
# output = f"../data/baseline_recommendations_for_{len(user_df)}_users.json"
output= f"../data/recommendations_for_{len(user_df)}_users_single_partition.json"
recommendations_df.to_json(output, index=False, orient="table")